In [1]:
import fiftyone as fo
import fiftyone.brain as fob
import pandas as pd
import os
import numpy as np
from glob import glob

In [2]:
# dataset name
name = "AIC_2024"

In [3]:
# run in about 15 seconds
if fo.dataset_exists(name):
    fo.delete_dataset(name)
    
dataset = fo.Dataset.from_images_dir(
    name=name, 
    images_dir="../data", 
    recursive=True
)

 100% |███████████| 106589/106589 [8.5s elapsed, 0s remaining, 12.6K samples/s]      


In [4]:
# if you want to automatically open the browser, just uncomment session.open_tab()
# or you can visit directly : http://localhost:5151/datasets/AIC_2024

session = fo.launch_app(dataset, auto=False)
# session.open_tab()

Session launched. Run `session.show()` to open the App in a cell output.


In [5]:
print(dataset.first())

<Sample: {
    'id': '66d20c8f06d76b7fd439824f',
    'media_type': 'image',
    'filepath': '/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/data/batch1/keyframes/keyframes_L01/L01_V001/001.jpg',
    'tags': [],
    'metadata': None,
}>


In [6]:
# run in about 36 seconds
unique_videos = set()
for sample in dataset:
    tmp, sample['video'], sample['keyframe_id'] = sample['filepath'][:-4].rsplit('/', 2)
    sample['batch'] = tmp.rsplit('/', 4)[-3]
    unique_videos.add(sample['video'])
    sample.save()

In [7]:
# map keyframes_id into corresponding frame_id
# run in nearly 40 seconds
video_frameid_dict = {}
for b in [1, 2, 3]:
    for video in unique_videos:
        filepath = f"../data/batch{b}/map-keyframes/{video}.csv"
        if os.path.exists(filepath):
            a = pd.read_csv(filepath)
            video_frameid_dict[video] = a['frame_idx']

for sample in dataset:
    sample['frame_id'] = video_frameid_dict[sample['video']].iloc[int(sample['keyframe_id']) - 1]
    sample.save()

In [8]:
print(dataset.first())

<Sample: {
    'id': '66d20c8f06d76b7fd439824f',
    'media_type': 'image',
    'filepath': '/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/data/batch1/keyframes/keyframes_L01/L01_V001/001.jpg',
    'tags': [],
    'metadata': None,
    'video': 'L01_V001',
    'keyframe_id': '001',
    'batch': 'batch1',
    'frame_id': 0,
}>


In [9]:
# load precomputed clip embeddings
# run in about 1 minutes

video_keyframe_dict = {}
all_keyframe = glob('../data/batch*/keyframes/*/*/*.jpg')

for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('/',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

embedding_dict = {}
for j in [1, 2, 3]:
    for video in unique_videos:
        clip_path = f"../data/batch{j}/clip-features-32/{video}.npy"
        if os.path.exists(clip_path):
            a = np.load(clip_path)
            embedding_dict[video] = {}
            for i, k in enumerate(video_keyframe_dict[video]):
                embedding_dict[video][k] = a[i]

clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['keyframe_id']]
    clip_embeddings.append(clip_embedding)

if dataset.has_brain_run("img_sim_clip"):
    dataset.delete_brain_run("img_sim_clip")
fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch",     
    embeddings=clip_embeddings,         
    brain_key="img_sim_clip",
)

In [10]:
# export the entire dataset into disk, see reload.ipynb for loading
dataset.export(
    export_dir="../export",
    dataset_type=fo.types.FiftyOneDataset,
    export_media=False
)

Directory '../export' already exists; export will be merged with existing files
Exporting samples...
 100% |██████████████| 106589/106589 [1.8s elapsed, 0s remaining, 69.8K docs/s]      
